In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:

import pandas as pd 
import numpy as np 
import os 
import glob
import plotly.express as px
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import bioread
from scipy.signal import resample

import pytz

from datetime import datetime
from pathlib import Path
import glob
import ipywidgets as widgets
from IPython.display import display

In [3]:
folder_path = '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File'
file_type = 'csv'  # Change to 'xlsx', 'json', etc., as needed.
file_pattern = os.path.join(folder_path, f'*.{file_type}')
files_concat = glob.glob(file_pattern)
files_concat

['/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/gry_P11.csv',
 '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/heart_rate_P7.csv',
 '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/heart_rate_P6.csv',
 '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/gry_P10.csv',
 '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/gry_P12.csv',
 '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/heart_rate_P4.csv',
 '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub

In [11]:
# Function to generate the figure
def plot_hr_raw_data(df, pid_str):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df["HeartRate"], mode="lines+markers"))
    fig.update_layout(title=f"Heart Rate Data for {pid_str}", xaxis_title="Timestamp", yaxis_title="Heart Rate")
    return fig

# def plot_hr_raw_data(df, pid_str):
#     fig = go.Figure()

#     # Create a stem plot by drawing vertical lines
#     for x, y in zip(df.index, df["HeartRate"]):
#         fig.add_trace(go.Scatter(x=[x, x], y=[0, y], mode="lines", line=dict(color='blue', width=1)))

#     # Add markers on top of the stems
#     fig.add_trace(go.Scatter(x=df.index, y=df["HeartRate"], mode="markers", marker=dict(color='red'), name=f"HR Data {pid_str}"))

#     fig.update_layout(
#         title=f"Heart Rate Data for {pid_str}",
#         xaxis_title="Timestamp",
#         yaxis_title="Heart Rate"
#     )

#     return fig

def acc_plots(acc_df):
    fig = go.Figure()
    # Create and style traces
    fig.add_trace(go.Scatter(y=acc_df.iloc[:, 1], name='Acc X',
                             line=dict(color='red', width=4)))
    fig.add_trace(go.Scatter(y=acc_df.iloc[:, 2], name = 'Acc Y',
                             line=dict(color='royalblue', width=4)))
    fig.add_trace(go.Scatter(y=acc_df.iloc[:, 3], name='Acc Z',
                             line=dict(color='green', width=4) # dash options include 'dash', 'dot', and 'dashdot'
    ))

    # Edit the layout
    fig.update_layout(title="Accelerometer Data",
                       xaxis_title='Number of Samples',
                       yaxis_title='Accelerometer Value')
    return fig

# Load files and extract PIDs.
files_concat = glob.glob("/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/*.csv")  
# Adjust the path as needed.
pid_to_file = {
    i.split("/")[-1].split("_")[-1].split(".")[0]: i 
    for i in files_concat if "task_timeline_heart_rate" in i
}

# Adjust the path as needed.
pid_to_file_acc = {
    i.split("/")[-1].split("_")[-1].split(".")[0]: i 
    for i in files_concat if "acc" in i
}
print("pid_to_file: ",pid_to_file)


pid_to_file:  {}


In [12]:
print("pid_to_file_acc json: ",pid_to_file_acc)

pid_to_file_acc json:  {'P16': '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/acc_P16.csv', 'P17': '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/acc_P17.csv', 'P4': '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/acc_P4.csv', 'P6': '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/acc_P6.csv', 'P15': '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/acc_P15.csv', 'P14': '/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/acc_P14.csv', 'P7': '/Users/shehjarsadhu/Desktop/Universit

In [13]:
sorted_pids = sorted(pid_to_file.keys(), key=lambda pid: int(pid[1:]))

# Ensure there's at least one file available
if not pid_to_file:
    raise ValueError("No CSV files found in the specified directory!")

# Create a dropdown menu with available PIDs
dropdown = widgets.Dropdown(
    options=[(pid, pid) for pid in sorted_pids],
    description="Select PID:",
    value=list(sorted_pids)[0],  # Set default selection
    style={"description_width": "initial"},
)

# Create a button to remove zeros
remove_zeros_button = widgets.Button(
    description="Remove Zeros",
    button_style="warning",
)
# Create a button to remove zeros
remove_resample_button = widgets.Button(
    description="Resample (1Hz)",
    button_style="warning",
)

# Output widget to display the plot
output = widgets.Output()

# State variable to track whether to filter zeros
remove_zeros = False
interpolate_hr = False

def toggle_remove_zeros(b):
    # Toggle the button's style between 'warning' and 'success'
    if b.button_style == "warning":
        b.button_style = "success"  # change to green
        global remove_zeros
        remove_zeros = not remove_zeros
    else:
        b.button_style = "warning"  # Change back to warning
        update_plot({"new": dropdown.value})

def toggle_resample_hr(b):
    global interpolate_hr
    interpolate_hr = not interpolate_hr
    update_plot({"new": dropdown.value})
    
# Attach the button click event
remove_zeros_button.on_click(toggle_remove_zeros)
remove_resample_button.on_click(toggle_resample_hr)

# Function to update the plot based on selection
def update_plot(change):
    output.clear_output(wait=True)
    selected_pid = change["new"]

    if selected_pid:
        df = pd.read_csv(pid_to_file[selected_pid])
#         df_acc = pd.read_csv(pid_to_file_acc[selected_pid],low_memory=False)

        # Sort the DataFrame by the 'time' column.
        df_sorted = df.sort_values(by='Timestamp_pd')
        df_reindexed = df_sorted.reset_index(drop=True)

        df_reindexed["HeartRate"] = pd.to_numeric(df_reindexed["HeartRate"], errors="coerce")

        # Apply zero filter if the toggle is active
        if remove_zeros:
            df_reindexed = df_reindexed[df_reindexed["HeartRate"] != 0]

        fig = plot_hr_raw_data(df_reindexed, selected_pid)

        fig.update_layout(
            xaxis=dict(
                title="Number of samples",
                showline=True,
                linecolor='black',
                linewidth=2,
            ),
            yaxis=dict(
                title="Heart Rate (bpm)",
                showline=True,
                linecolor='black',
                linewidth=2,
            ),
            title={
                'text': "Smart watch HR data in bpm",
                'x': 0.5,
                'xanchor': 'center',
                'y': 0.95,
                'yanchor': 'top',
                'pad': {'b': 10},
            }
        )
        fig2 = go.Figure() # temporary figure 2.

#         fig2 = acc_plots(df_acc) #plot_hr_raw_data(df_reindexed, selected_pid)

#         fig2.update_layout(
#             xaxis=dict(
#                 title="Time in seconds",
#                 showline=True,
#                 linecolor='black',
#                 linewidth=2,
#             ),
#             yaxis=dict(
#                 title="Heart Rate (bpm)",
#                 showline=True,
#                 linecolor='black',
#                 linewidth=2,
#             ),
#             title={
#                 'text': "Smart watch HR data in bpm",
#                 'x': 0.5,
#                 'xanchor': 'center',
#                 'y': 0.95,
#                 'yanchor': 'top',
#                 'pad': {'b': 10},
#             }
#         )
        # Create a subplot figure with 1 row and 2 columns
        combined_fig = make_subplots(rows=1, cols=2, subplot_titles=("Smartwatch HR data in bpm", 
                                                                     "Smartwatch Acc data in m/s^2"))

        # Add traces from fig1 to the left subplot (col=1)
        for trace in fig.data:
            combined_fig.add_trace(trace, row=1, col=1)

        # Add traces from fig2 to the right subplot (col=2)
        for trace in fig2.data:
            combined_fig.add_trace(trace, row=1, col=2)
        # Update layout of the combined figure
        combined_fig.update_layout(
#             title_text="Side by Side Plots",
            showlegend=False,
            xaxis1=dict(
                title="Time in seconds",
                showline=True,
                linecolor="black",
                linewidth=2,
            ),
            yaxis1=dict(
                title="Heart Rate (bpm)",
                showline=True,
                linecolor="black",
                linewidth=2,
            ),
            xaxis2=dict(
                title="Time in seconds",
                showline=True,
                linecolor="black",
                linewidth=2,
            ),
            yaxis2=dict(
                title="Heart Rate (bpm)",
                showline=True,
                linecolor="black",
                linewidth=2,
            ),
        )
        
        with output:
            display(combined_fig)

# Attach callback function to dropdown
dropdown.observe(update_plot, names="value")

# Display widgets
display(dropdown, remove_zeros_button, remove_resample_button, output)

# Trigger the initial plot manually
update_plot({"new": dropdown.value})


ValueError: No CSV files found in the specified directory!

## Grouped HR averages

In [14]:
# Function to calculate RMSSD safely
def calculate_rmssd(hr_bpm):
    # Drop NaN values
    hr_bpm = hr_bpm.dropna()
    
    # Convert HR (bpm) to RR intervals (ms)
    if len(hr_bpm) < 2:  # Avoid division by zero for empty groups
        return np.nan

    rr_intervals = [60000 / hr for hr in hr_bpm if hr > 0]  # Avoid division by zero
    
    if len(rr_intervals) < 2:  # RMSSD requires at least two RR intervals
        return np.nan

    diff_rr = np.diff(rr_intervals)  # Compute successive differences
    squared_diff = diff_rr ** 2  # Square the differences
    return np.sqrt(np.mean(squared_diff))  # Compute RMSSD

In [15]:

# Function to compute HRV metrics
def compute_hrv(hr_bpm):
    hr_bpm = hr_bpm.dropna()  # Remove NaN values
    
    if len(hr_bpm) < 2:  # Ensure at least two data points
        return pd.Series({'RMSSD': np.nan, 'SDNN': np.nan, 'pNN50': np.nan})
    
    # Convert HR (bpm) to RR intervals (ms)
    rr_intervals = [60000 / hr for hr in hr_bpm if hr > 0]  # Avoid division by zero
    
    if len(rr_intervals) < 2:
        return pd.Series({'RMSSD': np.nan, 'SDNN': np.nan, 'pNN50': np.nan})

    # Compute successive differences
    diff_rr = np.diff(rr_intervals)
    
    # RMSSD
    rmssd = np.sqrt(np.mean(diff_rr ** 2))
    
    # SDNN
    sdnn = np.std(rr_intervals, ddof=1)  # Sample standard deviation
    
    # pNN50
    nn50 = np.sum(np.abs(diff_rr) > 50)  # Count of RR interval differences > 50ms
    pnn50 = (nn50 / len(diff_rr)) * 100  # Percentage

    return pd.Series({'RMSSD': rmssd, 'SDNN': sdnn, 'pNN50': pnn50})


In [16]:

# # Example HR data in bpm (5 minutes of data at ~1 Hz sampling)
# heart_rate_data = [72, 75, 73, 70, 74, 76, 77, 72, 71, 69]
# rmssd_value = calculate_rmssd(heart_rate_data)

# print(f"RMSSD: {rmssd_value:.2f} ms")


In [17]:
# Create list of lists for pandas df. Each device and variable will have it's own list.
df_grouped_avg_list = [] # smartwatch
df_grouped_avg_biopac_list = [] # biopac
df_smartwatch_list_rmssd = []
df_biopac_list_rmssd = []
for sm_heart_rate in files_concat:
    if "heart_rate" in sm_heart_rate:
        df_sm_heart_rate = pd.read_csv(sm_heart_rate)
        cols = ['Stationary_Bike_Legs' , 'Rest_3' ,'Rest_1', 'Rest_2',
                'Give_Speech' , 'Prepare_Speech' ,'Stationary_Bike_Hand', 'Mental_Math']
        subset_df = df_sm_heart_rate[df_sm_heart_rate["activity"].isin(cols)]
        try:
            # Convert to float, setting errors='coerce' to replace errors with NaN
            subset_df['HeartRate_float_smartwatch'] = pd.to_numeric(subset_df['HeartRate'], errors='coerce')
            # Log how many problematic values were found
            num_invalid = subset_df['HeartRate_float_smartwatch'].isna().sum()
            if num_invalid > 0:
                print(f"Warning: {num_invalid} invalid heart rate values converted to NaN")
            #print("RESAMPLING ====================================================")
            # Drop NaN values before resampling (or you can impute if preferred)
            cleaned_df = subset_df.dropna(subset=['HeartRate_float_smartwatch'])
            resampled_df = resample_by_activity(cleaned_df, "HeartRate_float_smartwatch")
            #print(resampled_df.columns)
            #print(resampled_df["activity"].unique())
            #print(resampled_df.head())
           # print("resampled_df shape: \n", resampled_df.shape)
            print("========================================================================================")
        except Exception as e:
            print(f"Unexpected error: {e}")
            print("Filename with error for SMARTWATCH FLOAT VALUE CONVERSION",sm_heart_rate.split("/")[-1])

        try:
            # Apply RMSSD calculation grouped by 'subject'
            rmssd_results = subset_df.groupby('activity')['HeartRate_float_smartwatch'].apply(calculate_rmssd)
            # Convert to DataFrame if needed
            rmssd_df = rmssd_results.reset_index(name='RMSSD')
            rmssd_df["device"] ="smartwatch"
            rmssd_df["pid"] = sm_heart_rate.split("/")[-1].split("_")[-1].split(".")[0]
            #print("========================================================================================")
            #print("rmssd_df: ",rmssd_df)
            df_smartwatch_list_rmssd.append(rmssd_df)
        except ZeroDivisionError:
            print("ZeroDivisionError for RMSSD SMARTWATCH")
            print("File NAME with error for RMSSD",sm_heart_rate.split("/")[-1])
        grouped_avg = subset_df.groupby("activity")["HeartRate_float_smartwatch"].mean()
        df_grouped_avg = grouped_avg.to_frame()
        df_grouped_avg["activity"] = grouped_avg.index
        pid = sm_heart_rate.split("/")[-1].split("_")[-1].split(".")[0]
        df_grouped_avg["pid"] = pid
        df_grouped_avg_list.append(df_grouped_avg)
    if "biopac" in sm_heart_rate:
        print("BIOPAC filename",sm_heart_rate)
        df_biopac = pd.read_csv(sm_heart_rate)
        #print("BIOPAC: original file unique activities: ",df_biopac["activity"].unique())
      
       # print("==================================== Calculating Averages ====================================")
        #print("BIOPAC PARTICIPANT ID: ", sm_heart_rate.split("/")[-1].split("_")[0])
        #print("BIOPAC: Unique activities present in Biopac",df_biopac["activity"].unique())
        df_biopac['HeartRate_float_biopac'] = df_biopac['Heart_Rate'].astype(float)
        grouped_avg_biopac = df_biopac.groupby("activity")["HeartRate_float_biopac"].mean()
        #print("BIOPAC grouped_avg_biopac: ",grouped_avg_biopac)
        df_grouped_avg_biopac = grouped_avg_biopac.to_frame()
        df_grouped_avg_biopac["activity"] = grouped_avg_biopac.index
        pid = sm_heart_rate.split("/")[-1].split("_")[0]
        df_grouped_avg_biopac["pid"] = pid
        #print("BIOPAC grouped_avg_biopac:",df_grouped_avg_biopac)
        df_grouped_avg_biopac_list.append(df_grouped_avg_biopac)
#             ## ========================================================================================================================
        try:
            # Apply RMSSD calculation grouped by 'subject'
            rmssd_results = df_biopac.groupby('activity')['HeartRate_float_biopac'].apply(calculate_rmssd)
            # Convert to DataFrame if needed
            rmssd_df_biopac = rmssd_results.reset_index(name='RMSSD')
            rmssd_df_biopac["device"] ="biopac"
            pid = sm_heart_rate.split("/")[-1].split("_")[0]
            rmssd_df_biopac["pid"] = pid
            #print("========================================================================================")
            #print("rmssd_df biopac: ",rmssd_df_biopac)
            df_biopac_list_rmssd.append(rmssd_df_biopac)
        except ZeroDivisionError:
            print("ZeroDivisionError for RMSSD biopac")
        

/Users/shehjarsadhu/opt/anaconda3/envs/carehub/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Unexpected error: name 'resample_by_activity' is not defined
Filename with error for SMARTWATCH FLOAT VALUE CONVERSION heart_rate_P7.csv
Unexpected error: name 'resample_by_activity' is not defined
Filename with error for SMARTWATCH FLOAT VALUE CONVERSION heart_rate_P6.csv
Unexpected error: name 'resample_by_activity' is not defined
Filename with error for SMARTWATCH FLOAT VALUE CONVERSION heart_rate_P4.csv
BIOPAC filename /Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/P10_biopac.csv
Unexpected error: name 'resample_by_activity' is not defined
Filename with error for SMARTWATCH FLOAT VALUE CONVERSION heart_rate_P1.csv
BIOPAC filename /Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData_Drive_2024_25/Concat_File/P8_biopac.csv
BIOPAC filename /Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub_CareWear/DATASET/StudyData

In [ ]:
# Define mapping dictionary
activity_mapping = {
    'give speech': 'Give_Speech',
    'mental math': 'Mental_Math',
    'prepare speech': 'Prepare_Speech',
    'rest1': 'Rest_1',
    'rest2': 'Rest_2',
    'rest3': 'Rest_3',
    'stationary Bike1': 'Bike_Legs',
    'stationary Bike2': 'Bike_Hands'
}

# Apply mapping to the 'activity' column
df_biopac_rmssd = pd.concat(df_biopac_list_rmssd,ignore_index=True)
df_smartwatch_rmssd = pd.concat(df_smartwatch_list_rmssd,ignore_index=True)

# Apply mapping to the 'activity' column
df_biopac_rmssd['activity'] = df_biopac_rmssd['activity'].replace(activity_mapping)


In [ ]:
# Define mapping dictionary
activity_mapping = {
    'Rest_1':"Rest_1",
    'Prepare_Speech':"Prepare_Speech",
    'Give_Speech':"Give_Speech",
    'Rest_2':"Rest_2", 
    'Mental_Math':"Mental_Math",
    'Rest_3':"Rest_3",
    'Stationary_Bike_Legs':"Bike_Legs",
    'Stationary_Bike_Hand':"Bike_Hands"
}

# Apply mapping to the 'activity' column
df_smartwatch_rmssd['activity'] = df_smartwatch_rmssd['activity'].replace(activity_mapping)


In [ ]:
df_biopac_rmssd["activity"].value_counts()

In [ ]:

df_final = pd.concat(df_grouped_avg_list,ignore_index=True)
# Define mapping dictionary
activity_mapping = {
    'Rest_1':"Rest_1",
    'Prepare_Speech':"Prepare_Speech",
    'Give_Speech':"Give_Speech",
    'Rest_2':"Rest_2", 
    'Mental_Math':"Mental_Math",
    'Rest_3':"Rest_3",
    'Stationary_Bike_Legs':"Bike_Legs",
    'Stationary_Bike_Hand':"Bike_Hands"
}

# Apply mapping to the 'activity' column
df_final['activity'] = df_final['activity'].replace(activity_mapping)
print(df_final["activity"].value_counts())

In [ ]:
df_grouped_avg_biopac_list

In [ ]:

df_final_biopac = pd.concat(df_grouped_avg_biopac_list,ignore_index=True)
print(df_final_biopac["activity"].value_counts())

In [ ]:
# Define mapping dictionary
activity_mapping = {
    'give speech': 'Give_Speech',
    'mental math': 'Mental_Math',
    'prepare speech': 'Prepare_Speech',
    'rest1': 'Rest_1',
    'rest2': 'Rest_2',
    'rest3': 'Rest_3',
    'stationary Bike1': 'Bike_Legs',
    'stationary Bike2': 'Bike_Hands'
}

# Apply mapping to the 'activity' column
df_final_biopac['activity'] = df_final_biopac['activity'].replace(activity_mapping)

In [ ]:
df_final_biopac.head()
print(df_final_biopac["activity"].value_counts())


In [ ]:
#df_final_biopac.to_csv("~/Desktop/biopac_average_hr.csv")

In [ ]:
df_final_biopac["pid"].unique()

In [ ]:
df_final_biopac["pid"].unique()

In [ ]:
df_final_biopac["activity"].unique()

In [ ]:
custom_order_biopac = ['rest1', 'prepare speech', 'give speech', 'rest2', 
                        'Mental_Math', 'rest3',  'stationary Bike1','stationary Bike2']


In [ ]:
df_final_biopac

In [ ]:
# Define custom order
custom_order = ['Rest_1', 'Prepare_Speech', 'Give_Speech', 'Rest_2', 'Mental_Math', 'Rest_3', 'Bike_Legs', 'Bike_Hands']

# Convert 'activity' column to categorical with specified order
df_final['activity'] = pd.Categorical(df_final['activity'], categories=custom_order, ordered=True)
df_final_biopac['activity'] = pd.Categorical(df_final_biopac['activity'], categories=custom_order, ordered=True)

# Create traces separately for Smartwatch and Biopac
trace_smartwatch = go.Box(
    x=df_final["activity"],  # Activity on x-axis
    y=df_final["HeartRate_float_smartwatch"],  # Smartwatch HR on y-axis
    name="Smartwatch",  # Legend name,
    boxpoints='all', # represent all points
    marker=dict(color="#00439D")  # Blue color
)

trace_biopac = go.Box(
    x=df_final_biopac["activity"],  # Activity on x-axis
    y=df_final_biopac["HeartRate_float_biopac"],  # Biopac HR on y-axis
    name="Biopac",  # Legend name,
    boxpoints='all', # represent all points
    marker=dict(color="#FF5733")  # Orange color
)

# Create the figure and add both traces
fig = go.Figure(data=[trace_smartwatch, trace_biopac])

# Update layout
fig.update_layout(
    title=dict(
        text="Heart Rate Distribution by Activity",
        font=dict(size=24, color="black"),
        x=0.5,

        xanchor="center"
    ),
    xaxis=dict(
        title="Activity ID",
        title_font=dict(size=16, color="black"),
        tickfont=dict(size=14, color="black"),
        categoryorder="array",  # Ensure correct order
        categoryarray=custom_order
    ),
    yaxis=dict(
        title="Heart Rate Average",
        title_font=dict(size=16, color="black"),
        tickfont=dict(size=14, color="black")
    ),
    plot_bgcolor="rgba(240, 240, 240, 1)",  # Light gray background
    paper_bgcolor="rgba(230, 230, 230, 1)", # Slightly darker gray background
    boxmode="group"  # Ensures side-by-side grouping
)
# print("fsdfs")
# Show the figure
fig.show()

In [73]:
# df_biopac_rmssd = pd.concat(df_biopac_list_rmssd,ignore_index=True)
# df_smartwatch_rmssd = pd.concat(df_smartwatch_list_rmssd,ignore_index=True)
df_smartwatch_rmssd

,activity,RMSSD,device,pid
0,Give_Speech,7.996614,smartwatch,P7
1,Mental_Math,12.174510,smartwatch,P7
2,Prepare_Speech,10.166747,smartwatch,P7
3,Rest_1,7.883905,smartwatch,P7
4,Rest_2,10.997089,smartwatch,P7
...,...,...,...,...
75,Rest_2,5.162978,smartwatch,P8
76,Rest_3,6.982872,smartwatch,P8
77,Bike_Hands,7.951988,smartwatch,P8
78,Rest_2,27.964142,smartwatch,P9


In [74]:
df_biopac_rmssd

,activity,RMSSD,device,pid
0,give speech,24.356107,biopac,P10
1,mental math,16.740529,biopac,P10
2,prepare speech,28.161702,biopac,P10
3,rest1,21.783705,biopac,P10
4,rest2,17.852308,biopac,P10
...,...,...,...,...
105,rest1,3.746261,biopac,P13
106,rest2,12.469378,biopac,P13
107,rest3,15.133334,biopac,P13
108,stationary Bike1,5.362826,biopac,P13


In [101]:
# Define custom order
custom_order = ['Rest_1', 'Prepare_Speech', 'Give_Speech', 'Rest_2', 
                'Mental_Math', 'Rest_3', 'Bike_Legs', 'Bike_Hands']

# Convert 'activity' column to categorical with specified order
df_final['activity'] = pd.Categorical(df_smartwatch_rmssd['activity'], categories=custom_order, ordered=True)
df_final_biopac['activity'] = pd.Categorical(df_biopac_rmssd['activity'], categories=custom_order, ordered=True)

# Create traces separately for Smartwatch and Biopac
trace_smartwatch = go.Box(
    x=df_smartwatch_rmssd["activity"],  # Activity on x-axis
    y=df_smartwatch_rmssd["RMSSD"],  # Smartwatch HR on y-axis
    name="Smartwatch",  # Legend name,
    boxpoints='all', # represent all points
    marker=dict(color="#00439D")  # Blue color
)

trace_biopac = go.Box(
    x=df_biopac_rmssd["activity"],  # Activity on x-axis
    y=df_biopac_rmssd["RMSSD"],  # Biopac HR on y-axis
    name="Biopac",  # Legend name,
    boxpoints='all', # represent all points
    marker=dict(color="#FF5733")  # Orange color
)

# Create the figure and add both traces
fig = go.Figure(data=[trace_smartwatch, trace_biopac])

# Update layout
fig.update_layout(
    title=dict(
        text="Heart Rate (RMSSD (ms)) Distribution by Activity",
        font=dict(size=24, color="black"),
        x=0.5,

        xanchor="center"
    ),
    xaxis=dict(
        title="Activity ID",
        title_font=dict(size=16, color="black"),
        tickfont=dict(size=14, color="black"),
        categoryorder="array",  # Ensure correct order
        categoryarray=custom_order
    ),
    yaxis=dict(
        title="Heart Rate Average",
        title_font=dict(size=16, color="black"),
        tickfont=dict(size=14, color="black")
    ),
    plot_bgcolor="rgba(240, 240, 240, 1)",  # Light gray background
    paper_bgcolor="rgba(230, 230, 230, 1)", # Slightly darker gray background
    boxmode="group"  # Ensures side-by-side grouping
)
# print("fsdfs")
# Show the figure
fig.show()